In [100]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

In [101]:
SRC = '/Users/fathe/OneDrive/Documents/UK/MFF/Thesis/input'
LANDSCAPE = 1
VIDEO = 4

In [102]:
videos = [file for file in os.listdir(SRC) if file.endswith("MP4") and file.startswith("GX")]

In [103]:
videos

In [104]:
def getCorners(gray_frame, **feature_params):
    return cv2.goodFeaturesToTrack(gray_frame, **feature_params)

In [105]:
vidcap = cv2.VideoCapture(os.path.join(SRC, videos[VIDEO]))
frame = 100

In [106]:
vidcap.get(cv2.CAP_PROP_FPS)

In [107]:
vidcap.set(cv2.CAP_PROP_POS_FRAMES, frame)
status0, frame0 = vidcap.read()
status1, frame1 = vidcap.read()

In [108]:
frame0 = cv2.cvtColor(frame0, cv2.COLOR_BGR2GRAY)
frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
width = int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH) / 8)
height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT) / 8)
# frame0 = cv2.resize(frame0, (width, height))
# frame1 = cv2.resize(frame1, (width, height))

In [109]:
frame1.shape

In [110]:
plt.figure(figsize=(15,5))
ax = plt.subplot(121)
ax.imshow(frame0, cmap='gray')
ax.set_title("0")
ax = plt.subplot(122)
ax.imshow(frame1, cmap='gray')
ax.set_title("1")

In [111]:
lk_params = dict(winSize=(15, 15),
                     maxLevel=2,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
feature_params = dict(maxCorners=100,
                          qualityLevel=0.3,
                          minDistance=7,
                          blockSize=7)

In [112]:
corners = getCorners(frame0, **feature_params)
# brisk = cv2.BRISK_create()
# kp1, des1 = brisk.detectAndCompute(frame0,None)

In [113]:
corners.shape

In [114]:
p1, st, err = cv2.calcOpticalFlowPyrLK(frame0, frame1, corners, None, **lk_params)

In [115]:
corners.shape[0], np.sum(st)

In [116]:
frame0_copy = frame0.copy()
for corner in corners:
    x, y = corner[0]
    cv2.circle(frame0_copy, (int(x), int(y)), 20, (0, 255, 0), 3)
    
frame1_copy = frame1.copy()
for p in p1:
    x, y = p[0]
    cv2.circle(frame1_copy, (int(x), int(y)), 20, (0, 255, 0), 3)
    
plt.figure(figsize=(15,15))
ax = plt.subplot(121)
ax.imshow(frame0_copy, cmap='gray')
ax = plt.subplot(122)
ax.imshow(frame1_copy, cmap='gray')

In [117]:
movement_direction = np.mean(p1-corners, axis=0)

In [118]:
np.mean(err), np.std(err), np.max(err)

In [119]:
movement_direction

In [120]:
p0s = []
p1s = []
sts = []
errs = []
vidcap.set(cv2.CAP_PROP_POS_FRAMES, frame)

feature_params = dict(maxCorners=100,
                      qualityLevel=0.3,
                      minDistance=7,
                      blockSize=7)

lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

ret, old_frame = vidcap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

pbar = tqdm(total=2000, desc=f"Extraction motion direction")

while vidcap.get(cv2.CAP_PROP_POS_FRAMES) <= 2100:
    ret, frame = vidcap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners = getCorners(old_gray, **feature_params)

    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, corners, None, **lk_params)
    p0s.append(corners)
    p1s.append(p1)
    sts.append(st)
    errs.append(err)

    old_gray = frame_gray

    pbar.update(1)

vidcap.release()
pbar.close()

In [121]:
p0 = p0s[0]
p1 = p1s[0]
st = sts[0]

In [122]:
p0 = p0[st == 1]
p1 = p1[st == 1]

In [123]:
p0.shape, np.max(p0, axis=0)

In [124]:
p0[p0[:, LANDSCAPE-1] > 2000]

In [125]:
frame1.shape[LANDSCAPE]/3

In [126]:
upper_moves = []
center_moves = []
lower_moves = []
total_moves = []

for p0, p1, st in zip(p0s, p1s, sts):
    p0 = p0[st == 1]
    p1 = p1[st == 1]
    upper_points_p0 = p0[p0[:, LANDSCAPE-1] <= np.max(frame1.shape)/3]
    upper_points_p1 = p1[p0[:, LANDSCAPE-1] <= np.max(frame1.shape)/3]
    center_points_p0 = p0[(np.max(frame1.shape) / 3 < p0[:, LANDSCAPE-1]) & (p0[:, LANDSCAPE-1] <= 2 * np.max(frame1.shape) / 3)]
    center_points_p1 = p1[(np.max(frame1.shape) / 3 < p0[:, LANDSCAPE-1]) & (p0[:, LANDSCAPE-1] <= 2 * np.max(frame1.shape) / 3)]
    lower_points_p0 = p0[2*np.max(frame1.shape)/3 < p0[:, LANDSCAPE-1]]
    lower_points_p1 = p1[2*np.max(frame1.shape)/3 < p0[:, LANDSCAPE-1]]
    
    upper_move = np.mean(upper_points_p1 - upper_points_p0, axis=0)
    upper_moves.append(np.hstack((upper_move, upper_points_p0.shape[0])))
    center_move = np.mean(center_points_p1 - center_points_p0, axis=0)
    center_moves.append(np.hstack((center_move, center_points_p0.shape[0])))
    lower_move = np.mean(lower_points_p1 - lower_points_p0, axis=0)
    lower_moves.append(np.hstack((lower_move, lower_points_p0.shape[0])))
    total_move = np.mean(p1-p0, axis=0)
    total_moves.append(np.hstack((total_move, p0.shape[0])))

In [127]:
total_moves = np.asarray(total_moves)
np.mean(total_moves[total_moves[:, 2] > 0], axis=0), np.median(total_moves[total_moves[:, 2] > 0], axis=0), np.std(total_moves[total_moves[:, 2] > 0], axis=0), np.argmax(total_moves[total_moves[:, 2] > 0], axis=0), np.argmin(total_moves[total_moves[:, 2] > 0], axis=0)

In [140]:
total_moves[total_moves[:, 2] > 0][80]

In [141]:
total_moves[total_moves[:, 2] > 0][1154]

In [130]:
upper_moves = np.asarray(upper_moves)
np.mean(upper_moves[upper_moves[:, 2] > 0], axis=0), np.median(upper_moves[upper_moves[:, 2] > 0], axis=0), np.std(upper_moves[upper_moves[:, 2] > 0], axis=0), np.argmax(upper_moves[upper_moves[:, 2] > 0], axis=0), np.argmin(upper_moves[upper_moves[:, 2] > 0], axis=0)

In [131]:
upper_moves[upper_moves[:, 2] > 0][1823]

In [132]:
upper_moves[upper_moves[:, 2] > 0][730]

In [133]:
center_moves = np.asarray(center_moves)
np.mean(center_moves[center_moves[:, 2] > 0], axis=0), np.median(center_moves[center_moves[:, 2] > 0], axis=0), np.std(center_moves[center_moves[:, 2] > 0], axis=0), np.argmax(center_moves[center_moves[:, 2] > 0], axis=0), np.argmin(center_moves[center_moves[:, 2] > 0], axis=0)

In [134]:
center_moves[center_moves[:, 2] > 0][1485]

In [135]:
center_moves[center_moves[:, 2] > 0][1395]

In [136]:
lower_moves = np.asarray(lower_moves)
np.mean(lower_moves[lower_moves[:, 2] > 0], axis=0), np.median(lower_moves[lower_moves[:, 2] > 0], axis=0), np.std(lower_moves[lower_moves[:, 2] > 0], axis=0), np.argmax(lower_moves[lower_moves[:, 2] > 0], axis=0), np.argmin(lower_moves[lower_moves[:, 2] > 0], axis=0)

In [137]:
lower_moves[lower_moves[:, 2] > 0][1086]

In [138]:
lower_moves[lower_moves[:, 2] > 0][343]

In [139]:
np.save(os.path.join(SRC, videos[VIDEO]).replace(".MP4", "-upper_moves.npy"), upper_moves)
np.save(os.path.join(SRC, videos[VIDEO]).replace(".MP4", "-center_moves.npy"), center_moves)
np.save(os.path.join(SRC, videos[VIDEO]).replace(".MP4", "-lower_moves.npy"), lower_moves)
np.save(os.path.join(SRC, videos[VIDEO]).replace(".MP4", "-total_moves.npy"), total_moves)